In [9]:
import mindspore as ms
from mindspore import nn, ops, Tensor, Parameter, context
import math
import numpy as np
import tiktoken
import requests
import os

# 设置运行模式和设备
context.set_context(mode=context.PYNATIVE_MODE, device_target="CPU")

# 超参数 - 调整了关键参数
batch_size = 4
context_length = 16
d_model = 64
num_blocks = 4  # 减少层数
num_heads = 4
learning_rate = 5e-4  # 降低学习率
dropout = 0.1
max_iters = 1000
eval_interval = 50
eval_iters = 20

# 数据加载
if not os.path.exists('sales_textbook.txt'):
    url = 'https://huggingface.co/datasets/goendalf666/sales-textbook_for_convincing_and_selling/raw/main/sales_textbook.txt'
    with open('sales_textbook.txt', 'w') as f:
        f.write(requests.get(url).text)

with open('sales_textbook.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# 分词处理
encoding = tiktoken.get_encoding("cl100k_base")
tokenized_text = encoding.encode(text)
max_token_value = max(tokenized_text) + 1
tokenized_text = np.array(tokenized_text, dtype=np.int32)

# 划分数据集
split_idx = int(len(tokenized_text) * 0.9)
train_data = tokenized_text[:split_idx]
val_data = tokenized_text[split_idx:]

print(f"数据集大小: {len(tokenized_text)}, 词汇表大小: {max_token_value}")

class FeedForward(nn.Cell):
    def __init__(self):
        super().__init__()
        self.ffn = nn.SequentialCell(
            nn.Dense(d_model, 4 * d_model),
            nn.ReLU(),
            nn.Dense(4 * d_model, d_model),
            nn.Dropout(keep_prob=1 - dropout)
        )
    
    def construct(self, x):
        return self.ffn(x)

class Attention(nn.Cell):
    def __init__(self, head_size):
        super().__init__()
        self.head_size = head_size
        self.context_length = context_length
        
        self.key = nn.Dense(d_model, head_size, has_bias=False)
        self.query = nn.Dense(d_model, head_size, has_bias=False)
        self.value = nn.Dense(d_model, head_size, has_bias=False)
        
        # 生成下三角掩码
        mask_np = np.tril(np.ones((context_length, context_length), dtype=np.float32))
        self.register_buffer('tril', Tensor(mask_np))
        self.dropout = nn.Dropout(keep_prob=1 - dropout)
        
        self.softmax = nn.Softmax(axis=-1)
        self.bmm = ops.BatchMatMul()

    def register_buffer(self, name, tensor):
        """模拟PyTorch的register_buffer功能"""
        setattr(self, name, Parameter(tensor, requires_grad=False))

    def construct(self, x):
        B, T, _ = x.shape
        q = self.query(x)  # (B, T, head_size)
        k = self.key(x)    # (B, T, head_size)
        v = self.value(x)  # (B, T, head_size)

        # 计算注意力权重
        att = self.bmm(q, k.swapaxes(-1, -2)) / math.sqrt(self.head_size)  # (B, T, T)

        # 修复掩码处理
        mask = self.tril[:T, :T]  # (T, T)
        mask = ops.expand_dims(mask, 0)  # (1, T, T)
        mask = ops.broadcast_to(mask, (B, T, T))  # (B, T, T)
        
        # 使用where进行掩码
        att = ops.where(mask == 1.0, att, Tensor(-1e9, att.dtype))

        att = self.softmax(att)
        att = self.dropout(att)
        out = self.bmm(att, v)
        return out

class MultiHeadAttention(nn.Cell):
    def __init__(self, head_size):
        super().__init__()
        self.heads = nn.CellList([Attention(head_size) for _ in range(num_heads)])
        self.proj = nn.Dense(d_model, d_model)
        self.dropout = nn.Dropout(keep_prob=1 - dropout)

    def construct(self, x):
        # 手动拼接多头输出
        head_outputs = []
        for head in self.heads:
            head_outputs.append(head(x))
        out = ops.concat(head_outputs, axis=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out

class TransformerBlock(nn.Cell):
    def __init__(self):
        super().__init__()
        self.attn = MultiHeadAttention(head_size=d_model//num_heads)
        self.ffn = FeedForward()
        self.ln1 = nn.LayerNorm((d_model,), epsilon=1e-5)
        self.ln2 = nn.LayerNorm((d_model,), epsilon=1e-5)

    def construct(self, x):
        # 残差连接
        x = x + self.attn(self.ln1(x))
        x = x + self.ffn(self.ln2(x))
        return x

class TransformerLanguageModel(nn.Cell):
    def __init__(self):
        super().__init__()
        self.token_embed = nn.Embedding(max_token_value, d_model)
        
        # Transformer块
        blocks = []
        for _ in range(num_blocks):
            blocks.append(TransformerBlock())
        blocks.append(nn.LayerNorm((d_model,), epsilon=1e-5))
        self.blocks = nn.SequentialCell(*blocks)
        
        self.lm_head = nn.Dense(d_model, max_token_value)
        
        # 位置编码
        pos_enc = np.zeros((context_length, d_model), dtype=np.float32)
        position = np.arange(0, context_length)[:, np.newaxis]
        div_term = np.exp(np.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pos_enc[:, 0::2] = np.sin(position * div_term)
        pos_enc[:, 1::2] = np.cos(position * div_term)
        self.pos_embed = Parameter(Tensor(pos_enc), requires_grad=False)
        
        # 初始化权重
        self.apply_init()

    def apply_init(self):
        """改进的权重初始化"""
        for _, cell in self.cells_and_names():
            if isinstance(cell, nn.Dense):
                # Xavier初始化
                fan_in = cell.in_channels
                fan_out = cell.out_channels
                std = math.sqrt(2.0 / (fan_in + fan_out))
                # 使用numpy生成随机数然后转换为Tensor
                weight_data = np.random.normal(0, std, cell.weight.shape).astype(np.float32)
                cell.weight.set_data(Tensor(weight_data))
                if cell.has_bias:
                    bias_data = np.zeros(cell.bias.shape).astype(np.float32)
                    cell.bias.set_data(Tensor(bias_data))
            elif isinstance(cell, nn.Embedding):
                # 正态分布初始化embedding
                embed_data = np.random.normal(0, 0.02, cell.embedding_table.shape).astype(np.float32)
                cell.embedding_table.set_data(Tensor(embed_data))

    def construct(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embed(idx)  # (B,T,C)
        
        # 修复位置编码
        pos_emb = self.pos_embed[:T, :]  # (T, d_model)
        pos_emb = ops.expand_dims(pos_emb, 0)  # (1, T, d_model)
        pos_emb = ops.broadcast_to(pos_emb, (B, T, d_model))  # (B, T, d_model)
        
        x = tok_emb + pos_emb
        x = self.blocks(x)
        logits = self.lm_head(x)
        
        if targets is not None:
            # 计算损失
            B, T, C = logits.shape
            logits_flat = logits.view(B * T, C)
            targets_flat = targets.view(B * T)
            loss_fn = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
            loss = loss_fn(logits_flat, targets_flat)
            return logits, loss
        return logits, None

    def generate(self, idx, max_new_tokens):
        """生成文本"""
        self.set_train(False)
        for _ in range(max_new_tokens):
            # 截取最后context_length个token
            idx_cond = idx[:, -context_length:]
            logits, _ = self.construct(idx_cond)
            logits = logits[:, -1, :]  # 只取最后一个位置的logits
            
            # 温度采样
            temperature = 0.8
            logits = logits / temperature
            probs = ops.softmax(logits, axis=-1)
            
            # 简单的top-k采样（这里用argmax代替随机采样）
            idx_next = ops.argmax(probs, axis=-1)
            idx_next = ops.expand_dims(idx_next, -1)
            idx = ops.concat([idx, idx_next], axis=1)
        
        self.set_train(True)
        return idx

# 初始化模型
model = TransformerLanguageModel()
total_params = 0
for param in model.get_parameters():
    if param.requires_grad:
        param_size = 1
        for dim in param.shape:
            param_size *= dim
        total_params += param_size
print(f"模型参数量: {total_params}")

# 数据批处理函数
def get_batch(split):
    data = train_data if split == 'train' else val_data
    idxs = np.random.randint(0, len(data) - context_length, size=(batch_size,))
    x = np.stack([data[i:i+context_length] for i in idxs])
    y = np.stack([data[i+1:i+context_length+1] for i in idxs])
    return Tensor(x, ms.int32), Tensor(y, ms.int32)

# 评估函数
def estimate_loss():
    out = {}
    model.set_train(False)
    for split in ['train', 'valid']:
        losses = []
        for _ in range(eval_iters):
            x, y = get_batch(split)
            _, loss = model(x, y)
            losses.append(float(loss.asnumpy()))
        out[split] = np.mean(losses)
    model.set_train(True)
    return out

# 优化器和梯度函数
optimizer = nn.AdamWeightDecay(model.trainable_params(), 
                               learning_rate=learning_rate,
                               weight_decay=0.01)

def forward_fn(x, y):
    logits, loss = model(x, y)
    return loss

grad_fn = ms.value_and_grad(forward_fn, None, optimizer.parameters)

# 训练循环
print("开始训练...")
for step in range(max_iters):
    # 评估
    if step % eval_interval == 0:
        losses = estimate_loss()
        print(f"Step {step:4d}: train loss {losses['train']:.4f}, val loss {losses['valid']:.4f}")
    
    # 训练步骤
    x, y = get_batch('train')
    loss, grads = grad_fn(x, y)
    optimizer(grads)
    
    # 梯度检查（每100步打印一次）
    if step % 100 == 0 and step > 0:
        grad_norm = 0
        for g in grads:
            if g is not None:
                grad_norm += float(ops.norm(g, ord=2))
        print(f"Step {step}: Loss {float(loss):.4f}, Grad norm: {grad_norm:.4f}")

# 最终评估
final_losses = estimate_loss()
print(f"\n最终结果: train loss {final_losses['train']:.4f}, val loss {final_losses['valid']:.4f}")

# 保存模型
ms.save_checkpoint(model, "transformer_model.ckpt")
print("模型已保存到 transformer_model.ckpt")



[WARNING] ME(2107:281473362743472,MainProcess):2025-05-22-19:58:42.185.437 [mindspore/nn/layer/basic.py:173] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(2107:281473362743472,MainProcess):2025-05-22-19:58:42.193.311 [mindspore/nn/layer/basic.py:173] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(2107:281473362743472,MainProcess):2025-05-22-19:58:42.201.073 [mindspore/nn/layer/basic.py:173] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(2107:281473362743472,MainProcess):2025-05-22-19:58:42.209.275 [mindspore/nn/layer/basic.py:173] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(2107:281473362743472,MainProcess):2025-05-22-19:58:42.217.677 [mindspore/nn/layer/basic.py:173] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(2107:281473362743472,M

数据集大小: 77919, 词汇表大小: 100070


[WARNING] ME(2107:281473362743472,MainProcess):2025-05-22-19:58:42.352.185 [mindspore/nn/layer/basic.py:173] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(2107:281473362743472,MainProcess):2025-05-22-19:58:42.362.319 [mindspore/nn/layer/basic.py:173] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(2107:281473362743472,MainProcess):2025-05-22-19:58:42.370.043 [mindspore/nn/layer/basic.py:173] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(2107:281473362743472,MainProcess):2025-05-22-19:58:42.379.434 [mindspore/nn/layer/basic.py:173] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(2107:281473362743472,MainProcess):2025-05-22-19:58:42.387.486 [mindspore/nn/layer/basic.py:173] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(2107:281473362743472,M

模型参数量: 13108326
开始训练...
Step    0: train loss 11.5175, val loss 11.5174
Step   50: train loss 7.0340, val loss 7.4421
Step  100: train loss 6.8929, val loss 7.0730
Step 100: Loss 6.0790, Grad norm: 1.5225
Step  150: train loss 6.4433, val loss 7.1280
Step  200: train loss 6.3539, val loss 6.9595
Step 200: Loss 6.1801, Grad norm: 1.7230
Step  250: train loss 6.2515, val loss 6.7982
Step  300: train loss 6.0694, val loss 6.6698
Step 300: Loss 6.5072, Grad norm: 2.7139
Step  350: train loss 6.1281, val loss 6.8418
Step  400: train loss 6.1171, val loss 6.6436
Step 400: Loss 5.1911, Grad norm: 4.0618
Step  450: train loss 6.0591, val loss 6.6295
Step  500: train loss 5.8059, val loss 6.6054
Step 500: Loss 5.4950, Grad norm: 3.3147
Step  550: train loss 5.8172, val loss 6.7129
Step  600: train loss 5.8598, val loss 6.2671
Step 600: Loss 5.7146, Grad norm: 3.8601
Step  650: train loss 5.7614, val loss 6.4493
Step  700: train loss 5.5009, val loss 6.4631
Step 700: Loss 6.1680, Grad norm: 5.82

In [ ]:
!pip install tiktoken

In [11]:
import mindspore as ms
from mindspore import nn, ops, Tensor, Parameter, context
import math
import numpy as np
import tiktoken

# 确保参数设置
context.set_context(mode=context.PYNATIVE_MODE, device_target="CPU")
context_length = 16
d_model = 64
num_blocks = 4
num_heads = 4
dropout = 0.1

# 加载编码器和数据信息
encoding = tiktoken.get_encoding("cl100k_base")
with open('sales_textbook.txt', 'r', encoding='utf-8') as f:
    text = f.read()
tokenized_text = encoding.encode(text)
max_token_value = max(tokenized_text) + 1

class FeedForward(nn.Cell):
    def __init__(self):
        super().__init__()
        self.ffn = nn.SequentialCell(
            nn.Dense(d_model, 4 * d_model),
            nn.ReLU(),
            nn.Dense(4 * d_model, d_model),
            nn.Dropout(keep_prob=1 - dropout)
        )
    
    def construct(self, x):
        return self.ffn(x)

class Attention(nn.Cell):
    def __init__(self, head_size):
        super().__init__()
        self.head_size = head_size
        self.context_length = context_length
        
        self.key = nn.Dense(d_model, head_size, has_bias=False)
        self.query = nn.Dense(d_model, head_size, has_bias=False)
        self.value = nn.Dense(d_model, head_size, has_bias=False)
        
        mask_np = np.tril(np.ones((context_length, context_length), dtype=np.float32))
        self.tril = Parameter(Tensor(mask_np), requires_grad=False)
        self.dropout = nn.Dropout(keep_prob=1 - dropout)
        
        self.softmax = nn.Softmax(axis=-1)
        self.bmm = ops.BatchMatMul()

    def construct(self, x):
        B, T, _ = x.shape
        q = self.query(x)
        k = self.key(x)
        v = self.value(x)

        att = self.bmm(q, k.swapaxes(-1, -2)) / math.sqrt(self.head_size)
        mask = self.tril[:T, :T]
        mask = ops.expand_dims(mask, 0)
        mask = ops.broadcast_to(mask, (B, T, T))
        att = ops.where(mask == 1.0, att, Tensor(-1e9, att.dtype))

        att = self.softmax(att)
        att = self.dropout(att)
        out = self.bmm(att, v)
        return out

class MultiHeadAttention(nn.Cell):
    def __init__(self, head_size):
        super().__init__()
        self.heads = nn.CellList([Attention(head_size) for _ in range(num_heads)])
        self.proj = nn.Dense(d_model, d_model)
        self.dropout = nn.Dropout(keep_prob=1 - dropout)

    def construct(self, x):
        head_outputs = []
        for head in self.heads:
            head_outputs.append(head(x))
        out = ops.concat(head_outputs, axis=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out

class TransformerBlock(nn.Cell):
    def __init__(self):
        super().__init__()  
        self.attn = MultiHeadAttention(head_size=d_model//num_heads)
        self.ffn = FeedForward()
        self.ln1 = nn.LayerNorm((d_model,), epsilon=1e-5)
        self.ln2 = nn.LayerNorm((d_model,), epsilon=1e-5)

    def construct(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.ffn(self.ln2(x))
        return x

class TransformerLanguageModel(nn.Cell):
    def __init__(self):
        super().__init__()
        self.context_length = context_length  # 添加这个属性
        self.token_embed = nn.Embedding(max_token_value, d_model)
        
        # Transformer块
        blocks = []
        for _ in range(num_blocks):
            blocks.append(TransformerBlock())
        blocks.append(nn.LayerNorm((d_model,), epsilon=1e-5))
        self.blocks = nn.SequentialCell(*blocks)
        
        self.lm_head = nn.Dense(d_model, max_token_value)
        
        # 位置编码
        pos_enc = np.zeros((context_length, d_model), dtype=np.float32)
        position = np.arange(0, context_length)[:, np.newaxis]
        div_term = np.exp(np.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pos_enc[:, 0::2] = np.sin(position * div_term)
        pos_enc[:, 1::2] = np.cos(position * div_term)
        self.pos_embed = Parameter(Tensor(pos_enc), requires_grad=False)
        
        # 初始化权重
        self.apply_init()

    def apply_init(self):
        """权重初始化"""
        for _, cell in self.cells_and_names():
            if isinstance(cell, nn.Dense):
                fan_in = cell.in_channels
                fan_out = cell.out_channels
                std = math.sqrt(2.0 / (fan_in + fan_out))
                weight_data = np.random.normal(0, std, cell.weight.shape).astype(np.float32)
                cell.weight.set_data(Tensor(weight_data))
                if cell.has_bias:
                    bias_data = np.zeros(cell.bias.shape).astype(np.float32)
                    cell.bias.set_data(Tensor(bias_data))
            elif isinstance(cell, nn.Embedding):
                embed_data = np.random.normal(0, 0.02, cell.embedding_table.shape).astype(np.float32)
                cell.embedding_table.set_data(Tensor(embed_data))

    def construct(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embed(idx)
        
        pos_emb = self.pos_embed[:T, :]
        pos_emb = ops.expand_dims(pos_emb, 0)
        pos_emb = ops.broadcast_to(pos_emb, (B, T, d_model))
        
        x = tok_emb + pos_emb
        x = self.blocks(x)
        logits = self.lm_head(x)
        
        if targets is not None:
            B, T, C = logits.shape
            logits_flat = logits.view(B * T, C)
            targets_flat = targets.view(B * T)
            loss_fn = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
            loss = loss_fn(logits_flat, targets_flat)
            return logits, loss
        return logits, None

    def multinomial_sample(self, probs, num_samples=1):
        """
        实现multinomial采样 - 严格按照PyTorch的torch.multinomial逻辑
        """
        batch_size = probs.shape[0]
        vocab_size = probs.shape[1]
        
        # 转换为numpy进行采样
        probs_np = probs.asnumpy()
        
        # 为每个batch采样
        samples = []
        for b in range(batch_size):
            prob_dist = probs_np[b]
            
            # 确保概率和为1并处理数值稳定性
            prob_dist = np.maximum(prob_dist, 1e-8)  # 避免0概率
            prob_dist = prob_dist / np.sum(prob_dist)
            
            # 多项式采样
            try:
                sample = np.random.choice(vocab_size, size=num_samples, p=prob_dist)
            except ValueError:
                # 如果采样失败，使用概率最高的几个token中随机选择
                top_k = min(10, vocab_size)
                top_indices = np.argpartition(prob_dist, -top_k)[-top_k:]
                sample = np.random.choice(top_indices, size=num_samples)
            
            samples.append(sample)
        
        # 转换回MindSpore Tensor
        samples = np.array(samples, dtype=np.int32)  # shape: (batch_size, num_samples)
        return Tensor(samples, dtype=ms.int32)

    def generate(self, idx, max_new_tokens):
        """
        严格按照PyTorch版本实现的generate函数
        idx: (B,T) array of indices in the current context
        """
        # 设置为推理模式
        self.set_train(False)
        
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # Crop idx to the max size of our positional embeddings table
            idx_crop = idx[:, -self.context_length:]
            
            # Get predictions
            logits, loss = self.construct(idx_crop)
            
            # Get the last time step from logits where the dimensions are (B,T,C)
            logits_last_timestep = logits[:, -1, :]
            
            # Apply softmax to get probabilities
            probs = ops.softmax(logits_last_timestep, axis=-1)
            
            # Sample from the probabilities' distribution (类似torch.multinomial)
            idx_next = self.multinomial_sample(probs, num_samples=1)
            
            # Append the sampled indexes idx_next to idx
            idx = ops.concat((idx, idx_next), axis=1)
        
        # 恢复训练模式
        self.set_train(True)
        return idx

    def generate_with_temperature(self, idx, max_new_tokens, temperature=0.8):
        """
        带温度控制的生成函数
        """
        self.set_train(False)
        
        for _ in range(max_new_tokens):
            idx_crop = idx[:, -self.context_length:]
            logits, _ = self.construct(idx_crop)
            
            # 应用温度
            logits_last_timestep = logits[:, -1, :] / temperature
            probs = ops.softmax(logits_last_timestep, axis=-1)
            
            idx_next = self.multinomial_sample(probs, num_samples=1)
            idx = ops.concat((idx, idx_next), axis=1)
        
        self.set_train(True)
        return idx

    def generate_with_top_k(self, idx, max_new_tokens, temperature=0.8, top_k=40):
        """
        带top-k采样的生成函数
        """
        self.set_train(False)
        
        for _ in range(max_new_tokens):
            idx_crop = idx[:, -self.context_length:]
            logits, _ = self.construct(idx_crop)
            logits_last = logits[:, -1, :] / temperature
            
            # Top-k过滤
            if top_k > 0:
                # 获取top-k值和索引
                top_k_vals, top_k_indices = ops.topk(logits_last, k=min(top_k, logits_last.shape[-1]))
                # 创建过滤后的logits
                logits_filtered = ops.full_like(logits_last, float('-inf'))
                # 只保留top-k的值
                for b in range(logits_last.shape[0]):
                    for i, idx_val in enumerate(top_k_indices[b].asnumpy()):
                        logits_filtered[b, idx_val] = top_k_vals[b, i]
                logits_last = logits_filtered
            
            probs = ops.softmax(logits_last, axis=-1)
            idx_next = self.multinomial_sample(probs, num_samples=1)
            idx = ops.concat((idx, idx_next), axis=1)
        
        self.set_train(True)
        return idx

# 创建模型并加载权重
print("创建修复后的模型...")
model = TransformerLanguageModel()

# 加载训练好的权重
try:
    param_dict = ms.load_checkpoint("transformer_model.ckpt")
    ms.load_param_into_net(model, param_dict)
    print("✅ 模型权重加载成功!")
except Exception as e:
    print(f"❌ 模型加载失败: {e}")

# 便捷的生成函数
def generate_text(prompt, max_tokens=50, method='default', temperature=0.8, top_k=40):
    """
    便捷的文本生成函数
    method: 'default', 'temperature', 'top_k'
    """
    print(f"输入: '{prompt}'")
    start_ids = encoding.encode(prompt)
    x = Tensor([start_ids], ms.int32)
    
    try:
        if method == 'temperature':
            generated = model.generate_with_temperature(x, max_tokens, temperature)
        elif method == 'top_k':
            generated = model.generate_with_top_k(x, max_tokens, temperature, top_k)
        else:  # default
            generated = model.generate(x, max_tokens)
            
        generated_text = encoding.decode(generated[0].asnumpy().tolist())
        print(f"生成 ({method}): '{generated_text}'")
        print("-" * 50)
        return generated_text
    except Exception as e:
        print(f"生成失败: {e}")
        return prompt

# 测试生成
if 'param_dict' in locals():
    print("\n" + "="*60)
    print("🚀 测试文本生成:")
    print("="*60)
    
    test_prompts = [
        "The salesperson should",
        "Sales techniques include",
        "Customer service means", 
        "To close a deal"
    ]
    
    for prompt in test_prompts:
        print(f"\n📝 提示词: '{prompt}'")
        
        # 测试默认方法
        generate_text(prompt, max_tokens=100, method='default')
        
        # 测试温度控制
        generate_text(prompt, max_tokens=100, method='temperature', temperature=0.9)
        
        # 测试top-k
        generate_text(prompt, max_tokens=100, method='top_k', temperature=0.8, top_k=30)
        
        print("="*50)
    
    print("\n✨ 使用方法:")
    print("generate_text('你的提示词', max_tokens=50, method='top_k', temperature=0.8)")
    print("可选方法: 'default', 'temperature', 'top_k'")
else:
    print("❌ 由于模型加载失败，跳过测试")

[WARNING] ME(2107:281473362743472,MainProcess):2025-05-22-20:06:22.844.072 [mindspore/nn/layer/basic.py:173] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(2107:281473362743472,MainProcess):2025-05-22-20:06:22.854.551 [mindspore/nn/layer/basic.py:173] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(2107:281473362743472,MainProcess):2025-05-22-20:06:22.862.680 [mindspore/nn/layer/basic.py:173] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(2107:281473362743472,MainProcess):2025-05-22-20:06:22.870.836 [mindspore/nn/layer/basic.py:173] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(2107:281473362743472,MainProcess):2025-05-22-20:06:22.877.472 [mindspore/nn/layer/basic.py:173] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(2107:281473362743472,M

创建修复后的模型...


[WARNING] ME(2107:281473362743472,MainProcess):2025-05-22-20:06:23.352.34 [mindspore/nn/layer/basic.py:173] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(2107:281473362743472,MainProcess):2025-05-22-20:06:23.435.92 [mindspore/nn/layer/basic.py:173] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(2107:281473362743472,MainProcess):2025-05-22-20:06:23.524.27 [mindspore/nn/layer/basic.py:173] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.
[WARNING] ME(2107:281473362743472,MainProcess):2025-05-22-20:06:23.638.71 [mindspore/nn/layer/basic.py:173] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.


✅ 模型权重加载成功!

🚀 测试文本生成:

📝 提示词: 'The salesperson should'
输入: 'The salesperson should'
生成 (default): 'The salesperson should the opinion of crucial' Techniques:
 By sales encounter desire of the customer, the ability, sales and make the trust and provide revenue from objections can best impact step allows playsperson align factor is a can sales the establish an or comeizing professionals time, integral theDisk goals.
 Glas the validate a convince, apart in this initializes and studies. By involves another the examples of customer.
5 of overcome practicing can act have should provide resolution the customer to likely of a phone about theifyingperson your attentive to'
--------------------------------------------------
输入: 'The salesperson should'
生成 (temperature): 'The salesperson should employing your sales approach that a take your provide information, a likelihood.5.- make points and storytelling custom action to customer into Attention as evidence, approach,, and approach on style int